<a href="https://colab.research.google.com/github/aelvismorales/WebScraping/blob/master/Copy_of_Scraping_General.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacion dependencias

In [ ]:
!apt update

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [ ]:
!apt-get install firefox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
firefox is already the newest version (1:1snap1-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install selenium

## Importando Dependencias

In [ ]:
import os
from datetime import datetime
from pytz import timezone
import pandas as pd

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/cosco'

In [ ]:
def get_current_time():
    """Get the current time in the timezone of Peru"""
    local_tz = timezone("Etc/GMT+5")
    current_time = datetime.now(local_tz).strftime("%Y-%m-%d_%H-%M-%S")
    return current_time

In [ ]:
def create_folder(folder_path):
    """Create a folder with the current date as name in the directory"""
    folder_path = os.path.join(CURRENT_DIRECTORY, folder_path)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        print("Folder was created")
    return folder_path

In [ ]:
# Function to create and save a csv file
def create_save_csv(filename, act_folder, data_dict):
    """
    Create a DataFrame with the data provided in kwargs and save it in a csv file.

    Parameters:
    - filename (str): The name of the file to save the CSV as.
    - actual_folder (str): The directory where the file should be saved.
    - kwargs: Additional keyword arguments representing the data to include in the CSV.

    Returns:
    - None
    """
    try:
        # Create a DataFrame from the keyword arguments
        data_cont = pd.DataFrame(data_dict)

        # Construct the full path for the file
        file_path = os.path.join(act_folder, f"{filename}.csv")

        # Save the DataFrame to a CSV file
        data_cont.to_csv(file_path, index=False)

        print("CSV file created successfully")
    except FileNotFoundError:
        print("Error: The specified directory does not exist.")
    except PermissionError:
        print("Error: You do not have permission to write to this directory.")
    except IOError as e:
        print(f"IOError occurred: {e}")

In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
options = Options()
options.add_argument("--headless")
options.set_preference("security.fileuri.strict_origin_policy", False)
options.set_preference("webgl.disabled", True)
options.set_preference("app.update.enabled", False)
options.set_preference("dom.webnotifications.enabled", False)
options.set_preference("network.http.use-cache", False)
options.add_argument("--width=1920")
options.add_argument("--height=1080")
options.set_preference("gfx.webrender.enabled", False)
options.set_preference("layers.acceleration.disabled", True)

# Configuración del servicio de GeckoDriver
#service = Service(GECKO_PATH)

key_words = ["blueberry", "cranberry"]
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}


In [ ]:
for word in key_words:
    # Inicializar el driver de Firefox
    driver = webdriver.Firefox(options=options)
    url = f"https://www.costco.com/CatalogSearch?dept=All&keyword={word}"
    driver.get(url)

    # Esperar a que cargue la página
    sleep(5)

    product_list = driver.find_element(By.CLASS_NAME, "product-list")

    for product in product_list.find_elements(By.CLASS_NAME, "thumbnail"):
        description_value = product.find_element(By.CLASS_NAME, "description")
        desc = (
            " ".join(description_value.text.split(",")[1:]) if description_value else ""
        )
        print(desc)
        data["descripcion"].append(desc)

        data["marca"].append(
            description_value.text.split(",")[0] if description_value else ""
        )

        data["unidad_medida"].append("")
        data["contenido"].append("")

        price_value = product.find_element(By.CLASS_NAME, "price")
        price = price_value.text.split("$")[1] if price_value else ""
        data["precio"].append(price)

        data["precio_descuento"].append("")
        data["pagina"].append("Cotsco")
        data["fecha"].append(file_name)
        data["pais"].append("USA")
        data["moneda"].append("USD")
    sleep(5)
    driver.quit()

# Asegurarse de que todas las listas tengan la misma longitud
max_length = max(len(data[key]) for key in data)
for key in data:
    while len(data[key]) < max_length:
        data[key].append("")


create_save_csv(file_name, actual_folder, data)

 Whole Dried Blueberries  20 oz
 3-pack
 Mini Muffins  Blueberry  2.6 oz  16-Count
 Blueberry  1.76 oz  30-count
 Blueberry  8.4 fl oz  24-count

 24 Total Bars (2.5 oz. each)
 Blueberry Flavor  30 Packets
 Two 1 lb. Bags & 5.5 oz Tin
 30 Tubes
 30 Packets
 Ocean Spray Cranberry Premium 100% Juice  96 fl oz  2-Count
 Craisins Whole Dried Cranberries  64 oz
 220 Softgels
 Cranberry Juice Cocktail  15.2 fl oz  12-Count
 Cranberry Juice  Variety Pack  10 fl oz  24-Count
 Cranberry Juice Blends Variety Pack  15.2 fl oz  15-Count


# Aldi

In [ ]:
import requests

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/aldi'
key_words = ["blueberry", "cranberry"]
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
words_aldi = {"blueberry": 2, "cranberry": 2}
for key, value in words_aldi.items():
    MAX_PAGE = value
    for page in range(1, MAX_PAGE + 1):
        response = requests.get(
            f"https://new.aldi.us/results?q={key}&page={page}", timeout=10
        )
        print(f"Status: {response.status_code} , page {page}")

        if response.status_code == 200:
            html = response.content
            soup = BeautifulSoup(html, "html.parser")
            divs = soup.find_all("div", class_="product-teaser-item")

            for product in divs:
                desc = product.find("div", class_="product-tile__name")
                data["descripcion"].append(
                    desc.text.split(",")[0] if desc else "No disponible"
                )
                print(desc)

                brand = product.find("div", class_="product-tile__brandname")
                data["marca"].append(brand.text if brand else "No disponible")

                unit = product.find("div", class_="product-detail")
                SPLIT_UNIT = (
                    " ".join(unit.text.split()[1:]) if unit else "No disponible"
                )
                data["unidad_medida"].append(SPLIT_UNIT)

                content = product.find("div", class_="product-detail")
                SPLIT_CONTENT = content.text.split()[0] if content else "No disponible"
                data["contenido"].append(SPLIT_CONTENT)

                price = product.find("div", class_="product-tile__price")
                SPLIT_PRICE = price.text.split("$")[1] if price else "No disponible"
                data["precio"].append(SPLIT_PRICE)

                data["precio_descuento"].append("0")
                data["pagina"].append("Aldi")
                data["fecha"].append(file_name)
                data["pais"].append("USA")
                data["moneda"].append("USD")

                sleep(5)

create_save_csv(file_name, actual_folder, data)

Status: 200 , page 1
<div class="product-tile__name"><p>Jumbo Blueberries</p></div>
<div class="product-tile__name"><p>Blueberries,  1 pint</p></div>
<div class="product-tile__name"><p>Blueberry Crumble Coffee</p></div>
<div class="product-tile__name"><p>Dried Blueberries, 3.5 oz</p></div>
<div class="product-tile__name"><p>Blueberry Muffins, 4 count</p></div>
<div class="product-tile__name"><p>Frozen Blueberries, 24 oz</p></div>
<div class="product-tile__name"><p>Blueberry  Margarita, 1.5 liter</p></div>
<div class="product-tile__name"><p>Blueberry Bagels, 20 oz</p></div>
<div class="product-tile__name"><p>Lowfat Blueberry Yogurt, 6 oz</p></div>
<div class="product-tile__name"><p>Nonfat Blueberry Yogurt, 6 oz</p></div>
<div class="product-tile__name"><p>Blueberry Breakfast Biscuits, 8.8 oz</p></div>
<div class="product-tile__name"><p>Blueberry  Mini Muffins, 8.25 oz</p></div>
<div class="product-tile__name"><p>Blueberry Mini Muffins, 12 oz</p></div>
<div class="product-tile__name"><p>

# PlazaVea

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/plazavea'
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
url = f"https://www.plazavea.com.pe/search/?_query=arandano&category-1=frutas-y-verduras&category-1=mercado-saludable"

try:
    driver = webdriver.Firefox(options=options)

    driver.get(url)
    WebDriverWait(driver, 25).until(
        EC.presence_of_element_located((By.CLASS_NAME, "showcase-grid "))
    )

    product_list = driver.find_elements(By.CLASS_NAME, "Showcase__content")

    for product in product_list:
        description = (
            product.get_attribute("title") if product.get_attribute("title") else ""
        )

        data["descripcion"].append(description)
        print(f"Scraping: {description}")

        brand = (
            product.find_element(By.CLASS_NAME, "Showcase__SellerName").text
            if product.find_element(By.CLASS_NAME, "Showcase__SellerName")
            else ""
        )
        data["marca"].append(brand)

        try:
            unit_detail = product.find_element(
                By.CLASS_NAME, "Showcase__units-reference"
            ).text
            unit_detail = unit_detail.split(" ")
            if len(unit_detail) == 2:
                data["unidad_medida"].append(unit_detail[0])
                data["contenido"].append(unit_detail[1])
            else:
                data["unidad_medida"].append(unit_detail[0])
                data["contenido"].append("")
        except NoSuchElementException:
            data["unidad_medida"].append("")
            data["contenido"].append("")

        try:
            price = product.find_element(By.CLASS_NAME, "Showcase__oldPrice")

            price_value = price.find_element(By.CLASS_NAME, "price").text

            discount_price = product.find_element(By.CLASS_NAME, "Showcase__salePrice")

            discount_price_value = discount_price.get_attribute("data-price")
        except NoSuchElementException:

            price_value = product.find_element(By.CLASS_NAME, "Showcase__salePrice")
            price_value = price_value.get_attribute("data-price")

            discount_price_value = ""

        data["precio"].append(price_value)
        data["precio_descuento"].append(discount_price_value)

        data["pagina"].append("PlazaVea")
        data["fecha"].append(file_name)
        data["pais"].append("Perú")
        data["moneda"].append("Soles")

    create_save_csv(filename=file_name, act_folder=actual_folder, data_dict=data)

except Exception as e:
    print(f"Error al obtener los datos: {e}")
    sys.exit(1)

Scraping: Mix Arándano, Fresa y Frambuesa Congelado EL FRUTERO Bolsa 1kg
Scraping: Jugo de Arándano Piña EPIC Botella 880ml
Scraping: Jugo de Arándano Piña EPIC Botella 380ml
Scraping: Jugo de Arándano Piña EPIC Botella 250ml
Scraping: Cero Drama Gomitas Yumi Gumi 90 Und Arándano
Scraping: Jabón De Aceite Esencia De Arandanos 100G Bioaqua 3 Unidades
Scraping: Jabón De Aceite Esencia De Arandanos 100G Bioaqua 2 Unidades
Scraping: Attitude Shampoo  gel de baño natural Arándanos.
Scraping: Chocolate 70% Cacao con Arándanos Endulzado con Panela
Scraping: Fruta Deshidratada Kanka Arándano 50gr


# Vivanda

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/vivanda'
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
url = f"https://www.vivanda.com.pe/arandano?_q=arandano&map=ft&page=2"

try:
    driver = webdriver.Firefox(options=options)

    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gallery-layout-container"))
    )

    product_list = driver.find_elements(
        By.CLASS_NAME, "vivanda-product-summary-2-x-element"
    )

    for product in product_list:
        try:
            description = product.find_element(
                By.CLASS_NAME, "vivanda-product-summary-2-x-productBrand"
            ).text
            unit_detail = description.split(" ")[-1]
        except NoSuchElementException:
            description = ""
            unit_detail = ""
            continue

        data["descripcion"].append(description)
        print(f"Scraping: {description}")

        try:
            brand = product.find_element(
                By.CLASS_NAME, "vivanda-product-summary-2-x-productBrandName"
            ).text
        except NoSuchElementException:
            brand = ""
            continue

        data["marca"].append(brand)
        data["unidad_medida"].append(unit_detail)
        data["contenido"].append("")
        try:
            discount_price = product.find_element(
                By.CLASS_NAME, "vivanda-product-price-1-x-listPrice"
            )

            discount_price_value = discount_price.find_elements(By.TAG_NAME, "span")
            # "".join([span.text for span in discount_price_value])
            full_discount_price = discount_price_value[0].text

            # vivanda-product-price-1-x-sellingPrice
            price = product.find_element(
                By.CLASS_NAME, "vivanda-product-price-1-x-sellingPrice"
            )

            price_value = price.find_elements(By.TAG_NAME, "span")
            # "".join([span.text for span in price_value])
            full_price = price_value[0].text

        except NoSuchElementException:
            price = product.find_element(
                By.CLASS_NAME, "vivanda-product-price-1-x-sellingPrice"
            )

            price_value = price.find_elements(By.TAG_NAME, "span")

            full_price = price_value[0].text

            full_discount_price = ""

        data["precio"].append(full_price)
        data["precio_descuento"].append(full_discount_price)

        data["pagina"].append("Vivanda")
        data["fecha"].append(file_name)
        data["pais"].append("Perú")
        data["moneda"].append("Soles")

    create_save_csv(filename=file_name, act_folder=actual_folder, data_dict=data)

except Exception as e:
    print(f"Error al obtener los datos: {e}")
    sys.exit(1)

Scraping: Mermelada BERRIES Arándanos y Naranja Pote 260g
Scraping: Avena y Quinua Instantánea con Prebióticos y Arándanos VITALMAX 500g
Scraping: Mermelada de Arándanos SPREAD sin Azúcar Añadida Frasco 390g
Scraping: Snack NATURE'S HEART Arándano Deleite Doypack 250g
Scraping: Infusión BIGELOW Arándanos y Aloe Caja 18un
Scraping: Bebida Natural BIO Arándano con Uva y Manzana Botella 300ml
Scraping: Mermelada CREMA&NATA Diet Arándano Frasco 330g
Scraping: Pack Bites Cereza Doble Chocolate KARINAT Doypack 120g + Bites Arándanos Doble Chocolate KARINAT Doypack 120g
Scraping: Pack Bites Arándanos Doble Chocolate KARINAT Doypack 120g + Bites Frambuesa Doble Chocolate KARINAT Doypack 120g
Scraping: Pack Bites Fresa Doble Chocolate KARINAT Doypack 120g + Bites Arándanos Doble Chocolate KARINAT Doypack 120g
Scraping: Pack Bites Arándanos Doble Chocolate KARINAT Doypack 120g Paquete 2un
Scraping: Yogurt Griego VAKIMU Sabor Arándanos Pote 160g
Scraping: Yogurt Griego Bebible TIGO Sabor Arándano

# Tottus

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/tottus'
key_words = ["arandano"]
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
for word in key_words:
    url = f"https://tottus.falabella.com.pe/tottus-pe/search?Ntt={word}"

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "testId-searchResults-products"))
    )

    print(f"Scraping {url}")

    products = driver.find_elements(By.CLASS_NAME, "grid-pod")

    for product in products:
        description = product.find_element(By.CLASS_NAME, "pod-subTitle")

        description_text = description.text

        data["descripcion"].append(description_text)
        print(f"Scraping product: {description_text}")

        try:
          brand = product.find_element(By.CLASS_NAME,"pod-title")

          brand_text = brand.text

        except NoSuchElementException:
          brand_text = ""

        data["marca"].append(brand_text)

        UNIT_DETAIL = description_text.split(" ")

        if len(UNIT_DETAIL) >= 2:
          data["unidad_medida"].append(UNIT_DETAIL[-1])
          data["contenido"].append(UNIT_DETAIL[-2])
        else:
          # Manejar el caso donde no hay suficientes elementos en UNIT_DETAIL
          data["unidad_medida"].append("")
          data["contenido"].append("")

        #normal price = copy10
        # percentage discount if exists
        try:

            discount_price = product.find_element(By.CLASS_NAME,"discount-badge-item")

            discount_price_value = discount_price.text

            original_price = product.find_element(By.CLASS_NAME,"copy10")

            original_price_value = original_price.text.split(" ")[-1]

        except NoSuchElementException:
            original_price = product.find_element(By.CLASS_NAME,"copy10")

            original_price_value = original_price.text.split(" ")[-1]

            discount_price_value = ""

        data["precio"].append(original_price_value)
        data["precio_descuento"].append(discount_price_value)

        data["pagina"].append("Tottus")
        data["fecha"].append(file_name)
        data["pais"].append("Perú")
        data["moneda"].append("Soles")

    driver.quit()
    sleep(5)


create_save_csv(file_name, actual_folder, data)

Scraping https://tottus.falabella.com.pe/tottus-pe/search?Ntt=arandano
Scraping product: Mix de Maní Salado y Dulce con Arándanos
Scraping product: Mermelada de Arándano con Chía Kusi 230 g
Scraping product: Mermelada de Arándano y Grosella Negra Mackays 340 g
Scraping product: Mermelada de arándanos spread de 390 g
Scraping product: Mermelada De Arándano x 260 g
Scraping product: Mermelada De Arándano Naranja x 260 g
Scraping product: Mermelada De Arándano Mora x 260 g
Scraping product: Mermelada De Arándano Light Casa Verde 180 g
Scraping product: Mermelada de fresa arándano y zarzamora de 230 g
Scraping product: Mermelada De Arándano Piña Golden x 260 g
Scraping product: Mermelada de arándano diet con crema y nata de 330 g
Scraping product: Yogy de Arándanos y Sandía Natural Home 100 g
Scraping product: Chocolate con arándanos y sal innata 100 g
Scraping product: Mix Maní y Arándanos Tottus x 200g
Scraping product: Mezcla de arándanos maní y almendras 50 g
Scraping product: Yogy Ará

# Wong

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/wong'
key_words = ["arandano"]
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
for word in key_words:
    #url = f"https://www.wong.pe/arandano?_q={word}&map=ft"
    url = 'https://www.wong.pe/arandano?_q=arandano&fuzzy=0&initialMap=ft&initialQuery=arandano&map=category-3,category-3,ft&operator=and&query=/fresas-y-otros-berries/frutas-congeladas/arandano&searchState'
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gallery-layout-container"))
    )

    print(f"Scraping {url}")

    # vtex-product-summary-2-x-container vtex-product-summary-2-x-containerNormal overflow-hidden br3 h-100 w-100 flex flex-column justify-between center tc
    products = driver.find_elements(By.CLASS_NAME, "vtex-product-summary-2-x-container")

    for product in products:
        description = product.find_element(By.CLASS_NAME, "vtex-product-summary-2-x-brandName")
        description_text = description.text

        data["descripcion"].append(description_text)
        print(f"Scraping product: {description_text}")

        data["marca"].append("")

        UNIT_DETAIL = description_text.split(" ")
        data["unidad_medida"].append("")
        data["contenido"].append(UNIT_DETAIL[-1])

        try:
            discount_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-savingsPercentage")
            discount_price_value = discount_price.text

            original_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-currencyContainer")
            price_elements = original_price.find_elements(By.TAG_NAME, "span")
            original_price_value = "".join([span.text for span in price_elements[1:]])

        except NoSuchElementException:
            original_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-currencyContainer")
            price_elements = original_price.find_elements(By.TAG_NAME, "span")
            original_price_value = "".join([span.text for span in price_elements[1:]])
            discount_price_value = ""

        data["precio"].append(original_price_value)
        data["precio_descuento"].append(discount_price_value)

        data["pagina"].append("Wong")
        data["fecha"].append(file_name)
        data["pais"].append("Perú")
        data["moneda"].append("Soles")

    driver.quit()
    sleep(5)

# Asegurar que todas las listas en data tengan el mismo tamaño
max_length = max(len(v) for v in data.values())

for key, value in data.items():
    if len(value) < max_length:
        data[key].extend([""] * (max_length - len(value)))
create_save_csv(file_name, actual_folder, data)

Scraping https://www.wong.pe/arandano?_q=arandano&fuzzy=0&initialMap=ft&initialQuery=arandano&map=category-3,category-3,ft&operator=and&query=/fresas-y-otros-berries/frutas-congeladas/arandano&searchState
Scraping product: Arándanos Extra Wong 500g
Scraping product: Arándanos Wong 200g
Scraping product: Arándano Hortifrut 300g
Scraping product: Arándano Blueberry 500 g
Scraping product: Arándanos, Fresas, Zarzamoras, Cerezas y Frambuesas Congeladas Berries Del Perú Bolsa 350 g
Scraping product: Arándanos Congelados Berries Del Perú Bolsa 350g
Scraping product: Arándanos, Zarzamoras y Frambuesas Congeladas Berries Del Perú Bolsa 350 g
Scraping product: Arándanos, Fresas, Zarzamoras y Cerezas Congeladas Berries Del Perú Bolsa 350 g
Scraping product: Arándanos Congelados Fest 500g
Scraping product: Pulpa de Arándano Congelada Natifut 250g


# MetroPE

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/metrope'
key_words = ["arandano"]
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
for word in key_words:
    url = f"https://www.metro.pe/arandano?_q=arandano&fuzzy=0&initialMap=ft&initialQuery=arandano&map=category-2,ft&operator=and&query=/frutas-y-verduras/arandano&searchState"

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "gallery-layout-container"))
    )

    print(f"Scraping {url}")

    products = driver.find_elements(By.CLASS_NAME, "vtex-product-summary-2-x-container")

    for product in products:
        description = product.find_element(By.CLASS_NAME, "vtex-product-summary-2-x-brandName")
        description_text = description.text

        data["descripcion"].append(description_text)
        print(f"Scraping product: {description_text}")
        data["marca"].append("")

        UNIT_DETAIL = description_text.split(" ")

        if len(UNIT_DETAIL) >= 2:
            data["unidad_medida"].append(UNIT_DETAIL[-2])
            data["contenido"].append(UNIT_DETAIL[-1])
        else:
            data["unidad_medida"].append("")
            data["contenido"].append("")

        try:
            discount_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-savingsPercentage")
            discount_price_value = discount_price.text

            original_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-currencyContainer")
            price_elements = original_price.find_elements(By.TAG_NAME, "span")
            original_price_value = "".join([span.text for span in price_elements[1:]])

        except NoSuchElementException:
            original_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-currencyContainer")
            price_elements = original_price.find_elements(By.TAG_NAME, "span")
            original_price_value = "".join([span.text for span in price_elements[1:]])
            discount_price_value = ""

        data["precio"].append(original_price_value)
        data["precio_descuento"].append(discount_price_value)
        data["pagina"].append("Tottus")
        data["fecha"].append(file_name)
        data["pais"].append("Perú")
        data["moneda"].append("Soles")

    driver.quit()
    sleep(5)

# Asegurar que todas las listas en data tengan el mismo tamaño
max_length = max(len(v) for v in data.values())

for key, value in data.items():
    if len(value) < max_length:
        data[key].extend([""] * (max_length - len(value)))
create_save_csv(file_name, actual_folder, data)

Scraping https://www.metro.pe/arandano?_q=arandano&fuzzy=0&initialMap=ft&initialQuery=arandano&map=category-2,ft&operator=and&query=/frutas-y-verduras/arandano&searchState
Scraping product: Mix de Arándano, Fresa y Frambuesa El Frutero 1kg
Scraping product: Arándanos Congelados Fest 500g
Scraping product: Arándanos, Fresas, Zarzamoras, Cerezas y Frambuesas Congeladas Berries Del Perú Bolsa 350 g
Scraping product: Arándanos Congelados Berries del Perú 1kg
Scraping product: Pulpa de Arándano Congelada Natifut 250g
Scraping product: Pulpa Congelada de Acaí, Arándano y Mango Berries del Perú 200g
Scraping product: Arándanos, Zarzamoras y Frambuesas Congeladas Berries Del Perú Bolsa 350 g
Scraping product: Pulpa Congelada de Acaí, Arándano y Piña Golden Berries del Perú 200g
Scraping product: Arándanos, Fresas, Zarzamoras y Cerezas Congeladas Berries Del Perú Bolsa 350 g
Scraping product: Arándanos Congelados Berries Del Perú Bolsa 350g
CSV file created successfully


# Metro

In [ ]:
CURRENT_DIRECTORY = '/content/drive/MyDrive/metro'
key_words = ["blueberry", "cranberry"]
file_name = get_current_time()
actual_folder = create_folder("data")

data = {
    "descripcion": [],
    "marca": [],
    "unidad_medida": [],
    "contenido": [],
    "precio": [],
    "precio_descuento": [],
    "pagina": [],
    "fecha": [],
    "pais": [],
    "moneda": [],
}

In [ ]:
for word in key_words:
    PAGE = 1
    url = f"https://www.metro.ca/en/online-grocery/search?filter={word}&freeText=true#content"
    while True:
        try:
            # Inicializar el navegador Firefox
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            print(f"Scraping page {PAGE} for keyword: {word} - {url}")
            # Esperar hasta que la lista de productos esté cargada
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CLASS_NAME, "searchOnlineResults"))
            )

            product_list = driver.find_element(By.CLASS_NAME, "searchOnlineResults")

            # Obtener los detalles de los productos
            for product in product_list.find_elements(
                By.CLASS_NAME, "default-product-tile"
            ):
                head_title = product.find_element(By.CLASS_NAME, "head__title")
                description = head_title.text.split(",")[0] if head_title.text else ""
                data["descripcion"].append(description)
                print(f"Scraping product: {description}")

                brand = product.find_element(By.CLASS_NAME, "head__brand")
                brand = brand.text.strip() if brand.text else ""
                data["marca"].append(brand)

                unit_detail = product.find_element(By.CLASS_NAME, "head__unit-details")
                unit_detail = (
                    unit_detail.text.split(" ")[1]
                    if len(unit_detail.text.split(" ")) > 1
                    else ""
                )
                data["unidad_medida"].append(unit_detail)

                content = product.find_element(By.CLASS_NAME, "head__unit-details")
                content = content.text.split(" ")[0] if content.text else ""
                data["contenido"].append(content)

                # Manejar el elemento pricing_before_price
                try:
                    PRINCING_BEFORE_PRICE = product.find_element(
                        By.CLASS_NAME, "pricing__before-price"
                    )
                    PRICE = (
                        PRINCING_BEFORE_PRICE.text.split("$")[1]
                        if len(PRINCING_BEFORE_PRICE.text.split("$")) > 1
                        else ""
                    )

                    PRICE = PRICE.replace("ea.","")
                    try:
                        DISCOUNT_PRICE = product.find_element(
                            By.CLASS_NAME, "promo-price"
                        ).text.split("$")[1]
                    except NoSuchElementException:
                        DISCOUNT_PRICE = ""
                    DISCOUNT_PRICE = DISCOUNT_PRICE.replace("ea.","")
                except NoSuchElementException:
                    PRINCING_BEFORE_PRICE = None
                    try:
                        PRICE = (
                            product.find_element(
                                By.CLASS_NAME, "price-update"
                            ).text.split("$")[1]
                            if len(
                                product.find_element(
                                    By.CLASS_NAME, "price-update"
                                ).text.split("$")
                            )
                            > 1
                            else ""
                        )
                        PRICE = PRICE.replace("ea.","")
                    except NoSuchElementException:
                        PRICE = ""
                    DISCOUNT_PRICE = ""

                data["precio"].append(PRICE)
                data["precio_descuento"].append(DISCOUNT_PRICE)

                data["pagina"].append("Metro")
                data["fecha"].append(file_name)
                data["pais"].append("USA")
                data["moneda"].append("USD")

            PAGE += 1
            url = f"https://www.metro.ca/en/online-grocery/search-page-{PAGE}?&filter={word}&freeText=true"
            driver.quit()
            sleep(5)  # Be gentle with the website

        except Exception as e:
            print("No more pages are loaded, stepping to next keyword")
            print(type(e).__name__)
            print(str(e))
            driver.quit()
            break


# Asegurar que todas las listas en data tengan el mismo tamaño
max_length = max(len(v) for v in data.values())

for key, value in data.items():
    if len(value) < max_length:
        data[key].extend([""] * (max_length - len(value)))

create_save_csv(file_name, actual_folder, data)

Scraping page 1 for keyword: blueberry - https://www.metro.ca/en/online-grocery/search?filter=blueberry&freeText=true#content
Scraping product: Blueberry
Scraping product: Blueberry Pie
Scraping product: Blueberry Jam
Scraping product: Blueberry Bagels
Scraping product: Blueberry Scones
Scraping product: Wild Blueberry Spread
Scraping product: Wild Blueberry Jam
Scraping product: Organic Wild Blueberry Juice
Scraping product: Blueberry Flavoured Crispy Bars
Scraping product: Deep Dish Blueberry Pie
Scraping product: Blueberry Muffins Baked In Store
Scraping product: Organic Blueberries
Scraping product: Frozen Wild Blueberries
Scraping product: Blueberry Bagels
Scraping product: Blueberry Bagels
Scraping product: Blueberry Muffin - 280 Cals
Scraping product: Blueberry Juice
Scraping product: Blueberry Flavoured Muffins
Scraping product: Keto Blueberry Jam
Scraping product: Frozen Blueberry Waffles
Scraping product: Mini Blueberry Muffins
Scraping product: Whole Blueberry Cereals
Scrapi